In [ ]:
# Version mit vier Punkten in zwei Dimensionen, 
# und Kontrollpunkte im Decimal Format um Float Ungenauigkeit zu vermeiden. 
# Das Fitting findet aber weiter in float statt. Deshalb keine große Änderung.

# 1. Funktionen und Bibliotheken

# Löscht alle Variablen aus der aktuellen Sitzung um sicherzustellen, 
# dass keine alten Werte oder Definitionen den Code beeinflussen.
%reset -f

# Importieren der benötigten Bibliotheken
from decimal import Decimal, getcontext  # Für präzise Dezimalberechnungen
import numpy as np  # Für numerische Berechnungen
import pandas as pd  # Für die Arbeit mit Tabellen Datensätzen
import matplotlib.pyplot as plt # Für graphische Darstellung
from sklearn.model_selection import train_test_split  # Für das Teilen der Daten in Trainings- und Testsets
from sklearn.linear_model import LinearRegression  # Für die lineare Regressionsanalyse
from sklearn.metrics import mean_squared_error  # Für die Berechnung der mittleren quadratischen Abweichung (MSE)

# Globale Präzision für Decimal festlegen
getcontext().prec = 28

# Definition einer Funktion für die Berechnung von Punkten auf einer kubischen Bézierkurve
def cubic_bezier(points, t):
    # Punkte und Parameter in Decimal sicherstellen
    t = Decimal(t)
    p0, p1, p2, p3 = [np.array([Decimal(coord) for coord in point]) for point in points]
    # Berechnung der Bézierkurve
    return ((1 - t)**3 * p0 + 
            3 * (1 - t)**2 * t * p1 + 
            3 * (1 - t) * t**2 * p2 + 
            t**3 * p3)


# Funktion zur Generierung von 100 Punkten auf einer Bézierkurve
def generate_bezier_points(points, num_points=100):
    # Generiere `Decimal`-t-Werte
    t_values = [Decimal(t) for t in np.linspace(0, 1, num_points)]
    # Punkte auf der Bézierkurve generieren
    bezier_points = np.array([cubic_bezier(points, t) for t in t_values], dtype=object)
    return bezier_points


# Funktion zur Berechnung der Länge einer Kurve
def curve_length(curve):
    # Berechnet die Gesamtlänge einer Kurve basierend auf den Distanzen zwischen aufeinanderfolgenden Punkten
    distances = [point_distance(curve[i], curve[i+1]) for i in range(len(curve) - 1)]
    return sum(distances)

# Funktion zur Berechnung der Distanz zwischen zwei Punkten
def point_distance(p1, p2):
    # Umwandeln der Punkte in Decimal
    p1 = np.array([Decimal(coord) for coord in p1])
    p2 = np.array([Decimal(coord) for coord in p2])
    # Berechnung der euklidischen Distanz
    return Decimal(np.sqrt(np.sum((p1 - p2)**2)))

In [ ]:
# 2. Bézierkurven zufallsgenerieren und vermessen
from decimal import Decimal, getcontext

# Präzision für Decimal festlegen
getcontext().prec = 28

# Anzahl der Bézierkurven die erzeugt werden sollen
n = 1000

# Generierung von zufälligen Kontrollpunkten für n Bézierkurven
# Jede Bézierkurve hat 4 Kontrollpunkte im 2D-Raum, mit `Decimal`
punkte_cubic = np.random.randint(low=0, high=30, size=(n, 4, 2)).astype(Decimal)

# Initialisierung einer Liste zum Speichern der Ergebnisse
data = []

# Schleife über jede Gruppe von Kontrollpunkten
for i in range(n):
    # Kontrollpunkte in Decimal umwandeln
    control_points = [[Decimal(coord) for coord in point] for point in punkte_cubic[i]]
    
    # Generierung der Bézierkurve
    bezier_points = generate_bezier_points(control_points)
    
    # Berechnung der Länge der Bézierkurve
    length = curve_length(bezier_points)
    
    # Berechnung der Distanzen zwischen den Kontrollpunkten
    d01 = point_distance(control_points[0], control_points[1])
    d02 = point_distance(control_points[0], control_points[2])
    d03 = point_distance(control_points[0], control_points[3])
    d12 = point_distance(control_points[1], control_points[2])
    d13 = point_distance(control_points[1], control_points[3])
    d23 = point_distance(control_points[2], control_points[3])
    
    # Speichern der Ergebnisse als Eintrag in der Tabelle
    entry = {
        'P0': control_points[0],
        'P1': control_points[1],
        'P2': control_points[2],
        'P3': control_points[3],
        'Length': float(length),
        'D_P0_P1': float(d01),
        'D_P0_P2': float(d02),
        'D_P0_P3': float(d03),
        'D_P1_P2': float(d12),
        'D_P1_P3': float(d13),
        'D_P2_P3': float(d23),
    }
    data.append(entry)  # Hinzufügen des Eintrags

# Erstellen eines Pandas DataFrame aus der Ergebnissammlung
df_trainingsdaten = pd.DataFrame(data)

print("Die Kontrollpunkte der Kurven, Länge der Kurven, und Abstände zwischen den Kontrollpunkten:\n")
print(df_trainingsdaten)

In [ ]:
# 3. Plotten der ersten 15 zufallsgenerierten Bézierkurven zur Ansicht

fig, axes = plt.subplots(3, 5, figsize=(20, 12))  # 3x5 Raster für 15 Kurven
axes = axes.flatten()  # Umwandeln der Achsen in Eindimensionalität für einfachere Iteration

# Für die ersten 15 Bézierkurven
for idx, control_points in enumerate(punkte_cubic[:15]):  # Nur die ersten 15 Kontrollpunkt-Sets
    # Kontrollpunkte in Decimal umwandeln
    control_points = [[Decimal(coord) for coord in point] for point in control_points]
    bezier_curve = generate_bezier_points(control_points)  # Punkte auf der Bézierkurve generieren

    # Kontrollpunkte und Bézierkurve in NumPy-Arrays konvertieren
    bezier_curve = np.array(bezier_curve, dtype=float)  # Für das Plotten müssen es floats sein
    control_points = np.array(control_points, dtype=float)

    ax = axes[idx]
    
    # Plot der Bézierkurve
    ax.plot(bezier_curve[:, 0], bezier_curve[:, 1], color='blue')  # Blaue Bézierkurve
    # Kontrollpunkte plotten
    ax.plot(control_points[:, 0], control_points[:, 1], 'ro--', markersize=5)  # Rote Kontrollpunkte
    ax.set_title(f"Kurve {idx + 1}", fontsize=15)  
    ax.tick_params(axis='both', labelsize=15)  # Schriftgröße für die Achsenticks

# Entfernen überschüssiger Subplots, falls weniger als 15 Kurven vorhanden
for ax in axes[len(punkte_cubic[:15]):]:
    ax.axis('off')

# Globale Legende hinzufügen
fig.legend(
    ['Bézierkurve', 'Kontrollpunkte'], 
    loc='lower center', 
    ncol=2, 
    fontsize=20, 
    frameon=False
)

# Layout anpassen und anzeigen
plt.tight_layout(rect=[0, 0.05, 1, 1])  # Platz für die globale Legende unten schaffen
plt.show()

In [ ]:
# 4. Modell fitten und ausgeben 
# mit einigen Zusatzinformationen zur besseren Beurteilung

# Konvertiere die Längenspalte in Decimal
df_trainingsdaten['Length'] = df_trainingsdaten['Length'].apply(Decimal)
df_trainingsdaten['D_P0_P1'] = df_trainingsdaten['D_P0_P1'].apply(Decimal)
df_trainingsdaten['D_P0_P2'] = df_trainingsdaten['D_P0_P2'].apply(Decimal)
df_trainingsdaten['D_P0_P3'] = df_trainingsdaten['D_P0_P3'].apply(Decimal)
df_trainingsdaten['D_P1_P2'] = df_trainingsdaten['D_P1_P2'].apply(Decimal)
df_trainingsdaten['D_P1_P3'] = df_trainingsdaten['D_P1_P3'].apply(Decimal)
df_trainingsdaten['D_P2_P3'] = df_trainingsdaten['D_P2_P3'].apply(Decimal)

# Bestimmen und Ausgabe der kürzesten und längsten Länge, Durchschnitt und Median
min_length = df_trainingsdaten['Length'].min().quantize(Decimal('0.00001'))  # Kürzeste Länge
max_length = df_trainingsdaten['Length'].max().quantize(Decimal('0.00001'))  # Längste Länge
mean_length = (sum(df_trainingsdaten['Length']) / len(df_trainingsdaten['Length'])).quantize(Decimal('0.00001'))  # Durchschnitt
median_length = Decimal(df_trainingsdaten['Length'].median()).quantize(Decimal('0.00001'))  # Median

print(
    f"Längen der Kurven: [{min_length}, {max_length}]\n"
    f"Mittelwert: {mean_length}\n"
    f"Median: {median_length}\n"
)

# Für sklearn in float konvertieren
X = df_trainingsdaten[['D_P0_P1', 'D_P0_P2', 'D_P0_P3', 'D_P1_P2', 'D_P1_P3', 'D_P2_P3']].astype(float)
y = df_trainingsdaten['Length'].apply(float)

# Aufteilen der Daten in Trainings- und Testdatensätze
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Erstellen und Trainieren des Modells
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen mit dem Modell
y_pred = model.predict(X_test)

# Evaluierung des Modells
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {Decimal(mse).quantize(Decimal('0.00001'))}")  # Ausgabe der Abweichung

# Ausgabe der Modellparameter
coefficients = [Decimal(c).quantize(Decimal('0.00001')) for c in model.coef_]
intercept = Decimal(model.intercept_).quantize(Decimal('0.00001'))
print(f"Intercept: {intercept}\n")

# Labels für den Index erstellen
labels = ['a', 'b', 'c', 'd', 'e', 'f']

# Erstellen eines DataFrame und setzen der Labels als Index
coef_df = pd.DataFrame({
    'Variable': labels,
    'Gewicht': coefficients,
    'Feature': X.columns
})

# Ausgabe der Modellkoeffizienten
print("Gewichte der Linearen Regression:")
print(coef_df)

In [ ]:
# 5. Plotten der realen Werte vs. der Vorhersage

# Anlage des Diagramms
plt.figure(figsize=(10, 6)) 
plt.scatter(y_test, y_pred, alpha=0.6, color='blue', label='Datenpunkte')  # Streudiagramm

# Hinzufügen der Diagonalen für die Vorhersage
min_value = min(y_test.min(), y_pred.min(), 0)  # Kleinster Wert in den Daten
max_value = max(y_test.max(), y_pred.max())  # Größter Wert in den Daten
plt.plot([min_value, max_value], [min_value, max_value], color='red', linestyle='--', label='Vorhersage')

# Achsenbeschriftungen und Titel
plt.xlabel('Tatsächliche Werte (y_test)', fontsize=12)
plt.ylabel('Vorhergesagte Werte (y_pred)', fontsize=12)
plt.title('Berechnete vs. vorhergesagte Werte', fontsize=14)

# Hinzufügen einer Legende und eines Gitters
plt.legend()
plt.grid(True)

# Plot anzeigen
plt.show()

In [ ]:
# 6. Einführen und Plotten von Testdaten aus realen Projekten
# Siehe https://github.com/benjaminwand/cookie-cutters

punkte_testdaten = [
# corona virus   
[[35, 0], [35, 4.95], [33.9, 5.5], [32.8, 5.5]], 
[[23.835, 2.75], [31.7, 2.75], [29.5, 5.5], [32.8, 5.5]],
# chaosknoten    
[[36.0, 52.0], [60.0, 51.0], [65.0, 53.0], [70.0, 47.0]],
 [[36.0, 48.9], [60.0, 49.0], [65.0, 50.0], [69.0, 47.0]],
 [[77.0, 32.4], [75.5, 28.0], [72.0, 27.0], [71.7, 21.7]],
 [[79.0, 8.1], [72.5, 15.0], [82.0, 24.0], [77.0, 32.4]],
# cactus    
[[51, 81.5], [58, 81.5], [58.5, 63], [58.5, 58]], 
[[28, 0], [20, 0], [18.5, 2.8], [18.5, 3.2]],
[[18.5, 3.2], [18.5, 4], [20.5, 10], [20.5, 24]],
[[0, 52], [0, 65], [2.5, 75.5], [9.5, 75.5]],
[[9.5, 75.5], [19.5, 75.5], [11, 46], [17.2, 46]],
[[17.2, 46], [23, 46], [14, 90], [30, 90]],
# fairydust rocket
[[20, 0], [20, 14.5], [20, 21], [7.8, 28.2]],
[[33.7, 0], [33.7, 25], [32, 43], [19.5, 51]],
[[19.5, 51], [25, 88], [15, 97], [0, 103.8]]
]

# Testdaten in Decimal umwandeln
punkte_testdaten_decimal = [
    [[Decimal(coord) for coord in point] for point in curve]
    for curve in punkte_testdaten
]

# Plotten der Bézierkurven aus den punkte_testdaten
fig, axes = plt.subplots(3, 5, figsize=(20, 12))  # 3x5 Raster für 15 Kurven
axes = axes.flatten()  # Achsen in flache Liste umwandeln

# Plot jeder Bézierkurve
for idx, control_points in enumerate(punkte_testdaten_decimal):
    # Bézierkurve berechnen (Annahme: generate_bezier_points akzeptiert Decimal)
    bezier_curve = generate_bezier_points(control_points)
    
    # Kontrollpunkte und Bézierkurve in float für das Plotting umwandeln
    control_points_float = [[float(coord) for coord in point] for point in control_points]
    bezier_curve_float = [[float(coord) for coord in point] for point in bezier_curve]
    
    ax = axes[idx]
    # Bézierkurve plotten
    ax.plot(
        [point[0] for point in bezier_curve_float],
        [point[1] for point in bezier_curve_float],
        color="blue"
    )
    # Kontrollpunkte plotten
    ax.plot(
        [point[0] for point in control_points_float],
        [point[1] for point in control_points_float],
        'ro--',
        markersize=5
    )
    # Titel für jede Kurve
    ax.set_title(f"Bézierkurve {idx + 1}", fontsize=15)
    ax.tick_params(axis='both', labelsize=15)  # Schriftgröße für die Achsenticks

# Überschüssige Subplots deaktivieren (falls weniger als 15 Kurven)
for ax in axes[len(punkte_testdaten):]:
    ax.axis('off')

# Globale Legende hinzufügen
fig.legend(
    ['Bézierkurve', 'Kontrollpunkte'], 
    loc='lower center', 
    ncol=2, 
    fontsize=20, 
    frameon=False
)

# Layout anpassen und anzeigen
plt.tight_layout(rect=[0, 0.05, 1, 1])  # Platz für die globale Legende unten schaffen
plt.show()

In [ ]:
# 7. Berechnung und Ausgabe der Abstände zwischen den Kontrollpunkten für den Testdatensatz

# Definieren der Features, wie sie in den Trainingsdaten verwendet wurden
features = ['D_P0_P1', 'D_P0_P2', 'D_P0_P3', 'D_P1_P2', 'D_P1_P3', 'D_P2_P3']  

# Koeffizienten und Intercept aus dem vorhin trainierten Modell
coefficients = model.coef_  
intercept = model.intercept_  

# Berechnung der Testdaten mit Decimal
test_daten = []
for control_points in punkte_testdaten:
    # Kontrollpunkte in Decimal umwandeln
    control_points_decimal = [[Decimal(coord) for coord in point] for point in control_points]
    
    # Generierung der Bézierkurve mit Decimal
    bezier_points = generate_bezier_points(control_points_decimal)
    length = curve_length(bezier_points)  # Berechnung der Länge der Bézierkurve in Decimal
    
    # Berechnung der Abstände zwischen Kontrollpunkten
    d01 = point_distance(control_points_decimal[0], control_points_decimal[1])
    d02 = point_distance(control_points_decimal[0], control_points_decimal[2])
    d03 = point_distance(control_points_decimal[0], control_points_decimal[3])
    d12 = point_distance(control_points_decimal[1], control_points_decimal[2])
    d13 = point_distance(control_points_decimal[1], control_points_decimal[3])
    d23 = point_distance(control_points_decimal[2], control_points_decimal[3])
    
    # Speichern der Ergebnisse als Eintrag
    entry = {
        'D_P0_P1': d01,
        'D_P0_P2': d02,
        'D_P0_P3': d03,
        'D_P1_P2': d12,
        'D_P1_P3': d13,
        'D_P2_P3': d23,
        'Length': length,
    }
    test_daten.append(entry)

# Umwandeln der Testdaten in ein DataFrame
# Hier Decimal in float oder str konvertieren, da pandas nicht mit Decimal umgehen kann
df_test = pd.DataFrame([
    {k: float(v) for k, v in entry.items()} for entry in test_daten
])

# Berechnung und Hinzufügen der Vorhersagen für jede Zeile im DataFrame df_test
df_test['Predicted_Length'] = (
    df_test[features].dot(coefficients) + intercept
)

# Ausgabe des DataFrame
print("Die Kontrollpunkte der Kurven, Länge der Kurven, und Abstände zwischen den Kontrollpunkten:\n")
print(df_test)

In [ ]:
# 8. Ausgabe der mittleren quadratischen Abweichung 
# mit einigen Zusatzinformationen zur besseren Beurteilung

# Bestimmen und Ausgabe der kürzesten und längsten Länge
min_length = df_test['Length'].round(4).min()    # Kürzeste Länge
max_length = df_test['Length'].round(4).max()    # Längste Länge
mean_length = df_test['Length'].mean().round(4)  # Durchschnitt
median_length = df_test['Length'].round(4).median()  # Median

print(
    f"Längen der Kurven: [{min_length}, {max_length}]\n"
    f"Mittelwert: {mean_length}\n"
    f"Median: {median_length}\n"
)

# Vergleich der vorhergesagten und tatsächlichen Längen
mse = mean_squared_error(df_test['Length'], df_test['Predicted_Length'])
print(f"Mean Squared Error (MSE): {mse}")

In [ ]:
# 9. Erstellen des Plots, der Vorhersage und Testdaten gegenüberstellt

plt.figure(figsize=(10, 6))
plt.scatter(df_test['Length'], df_test['Predicted_Length'], color='blue', alpha=0.6, label='Berechnete Werte')

# Hinzufügen der Diagonalen für perfekte Vorhersage
max_value = max(df_test['Length'].max(), df_test['Predicted_Length'].max())
min_value = min(df_test['Length'].min(), df_test['Predicted_Length'].min(), 0)
plt.plot([min_value, max_value], [min_value, max_value], color='red', linestyle='--', label='Vorhersage')

# Achsenbeschriftungen und Titel
plt.xlabel('Tatsächliche Länge (Length)', fontsize=12)
plt.ylabel('Vorhergesagte Länge (Predicted Length)', fontsize=12)
plt.title('Vergleich: Tatsächliche vs. vorhergesagte Länge', fontsize=14)
plt.legend()
plt.grid(True)

# Plot anzeigen
plt.show()